In [188]:
!pip install openai requests pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ZarifBin.MohdZamrin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from openai import OpenAI
import json
from datetime import datetime, timedelta
import pandas as pd

# 🔑 Put your real key here
OPENAI_API_KEY = ""

client = OpenAI(api_key=OPENAI_API_KEY)
print("Client initialized.")


Client initialized.


In [190]:
def generate_schedule(constraints, events):
    """
    30-minute scheduler supporting:
    - fixed events (have a 'time' field)
    - flexible events (have flexible=True)
    - auto-placement to avoid conflicts
    """

    start = datetime.strptime(constraints["start_time"], "%H:%M")
    end   = datetime.strptime(constraints["end_time"], "%H:%M")

    # Build empty 30-min slots
    slots = {}
    current = start
    while current < end:
        t_str = current.strftime("%H:%M")
        slots[t_str] = None
        current += timedelta(minutes=30)

    # Split events into fixed vs flexible
    fixed_events = [e for e in events if "time" in e]
    flexible_events = [e for e in events if e.get("flexible")]

    # 1️⃣ PLACE FIXED EVENTS
    for e in fixed_events:
        name = e["name"]
        start_str = e["time"]
        dur = float(e.get("duration_hours", 1.0))

        try:
            event_start = datetime.strptime(start_str, "%H:%M")
        except:
            continue

        num_blocks = int(dur * 2)
        for i in range(num_blocks):
            slot_time = event_start + timedelta(minutes=30 * i)
            if slot_time < start or slot_time >= end:
                continue

            t_str = slot_time.strftime("%H:%M")
            existing = slots[t_str]

            if existing is None:
                slots[t_str] = name
            else:
                # Mark conflict
                if str(existing).startswith("CONFLICT:"):
                    if name not in existing:
                        slots[t_str] = existing + f" + {name}"
                elif existing != name:
                    slots[t_str] = f"CONFLICT: {existing} + {name}"

    # 2️⃣ PLACE FLEXIBLE EVENTS (AUTO-FIT)
    for e in flexible_events:
        name = e["name"]
        dur = int(float(e.get("duration_hours", 1.0)) * 2)  # 2 blocks per hour
        placed = False

        keys = sorted(slots.keys(), key=lambda s: datetime.strptime(s, "%H:%M"))

        for i in range(len(keys) - dur + 1):
            block = keys[i:i+dur]
            if all(slots[t] == "FREE TIME" or slots[t] is None for t in block):
                for t in block:
                    slots[t] = name
                placed = True
                break

        if not placed:
            print(f"WARNING: Could not place flexible event '{name}'")

    # 3️⃣ FILL EMPTY SLOTS WITH FREE TIME
    for k, v in slots.items():
        if v is None:
            slots[k] = "FREE TIME"

    # 4️⃣ BUILD FINAL SCHEDULE
    schedule = []
    for t, task in sorted(slots.items(),
                          key=lambda s: datetime.strptime(s[0], "%H:%M")):
        schedule.append({"time": t, "description": task})

    return schedule
def get_cat_fact():
    """Safe cat fact tool."""
    try:
        data = requests.get("https://catfact.ninja/fact", timeout=5).json()
        return {"fact": data.get("fact", "Cats are mysterious and awesome.")}
    except:
        return {"fact": "Cats are mysterious and awesome."}

def analyze_overlaps(constraints, events):
    """
    Analyze fixed events (those with a 'time' field) for overlaps.
    If overlaps exist, suggest alternative times for the affected events.

    Returns:
    {
        "has_conflict": bool,
        "conflicts": [
            {
                "events": ["Gym", "Eat"],
                "times": ["19:00", "19:30"]
            },
            ...
        ],
        "suggestions": [
            {
                "event": "Eat",
                "current_time": "19:00",
                "duration_hours": 1.0,
                "suggested_time": "20:00"
            },
            ...
        ]
    }
    """

    start = datetime.strptime(constraints["start_time"], "%H:%M")
    end   = datetime.strptime(constraints["end_time"], "%H:%M")

    # Build time slots (30-min) for occupancy tracking
    slot_times = []
    current = start
    while current < end:
        slot_times.append(current.strftime("%H:%M"))
        current += timedelta(minutes=30)

    # Map slot -> set of event indices occupying it
    occupancy = {t: set() for t in slot_times}

    # Only fixed events (those with explicit time) can overlap
    fixed_indices = []
    for idx, e in enumerate(events):
        if "time" not in e:
            continue
        fixed_indices.append(idx)
        name = e.get("name", f"Event {idx}")
        start_str = e["time"]
        dur = float(e.get("duration_hours", 1.0))

        try:
            ev_start = datetime.strptime(start_str, "%H:%M")
        except ValueError:
            continue

        num_blocks = int(dur * 2)
        for i in range(num_blocks):
            t = ev_start + timedelta(minutes=30 * i)
            if t < start or t >= end:
                continue
            t_str = t.strftime("%H:%M")
            if t_str in occupancy:
                occupancy[t_str].add(idx)

    # Find conflicts
    conflicts = []
    for t_str, idx_set in occupancy.items():
        if len(idx_set) > 1:
            event_names = sorted({events[i].get("name", f"Event {i}") for i in idx_set})
            conflicts.append({
                "events": event_names,
                "times": [t_str]
            })

    # Merge conflicts with same event pair across multiple times
    merged = {}
    for c in conflicts:
        key = tuple(c["events"])
        merged.setdefault(key, [])
        merged[key].extend(c["times"])

    conflict_list = [
        {"events": list(k), "times": sorted(v)}
        for k, v in merged.items()
    ]

    if not conflict_list:
        return {
            "has_conflict": False,
            "conflicts": [],
            "suggestions": []
        }

    # Build suggestions for each *fixed* event involved in conflicts
    suggestions = []
    for idx in fixed_indices:
        e = events[idx]
        name = e.get("name", f"Event {idx}")
        start_str = e.get("time")
        dur = float(e.get("duration_hours", 1.0))
        num_blocks = int(dur * 2)

        # Check if this event ever participates in a conflict
        participates = any(
            name in conf["events"] for conf in conflict_list
        )
        if not participates:
            continue

        # Try to find an alternative slot for this event
        orig_start = None
        try:
            orig_start = datetime.strptime(start_str, "%H:%M")
        except:
            pass

        suggested_time = None
        for candidate in slot_times:
            cand_start = datetime.strptime(candidate, "%H:%M")
            # Don't suggest the same start time
            if orig_start and cand_start == orig_start:
                continue

            # candidate range
            candidate_ok = True
            for i in range(num_blocks):
                t = cand_start + timedelta(minutes=30 * i)
                if t < start or t >= end:
                    candidate_ok = False
                    break
                t_str = t.strftime("%H:%M")
                # Allowed if empty or only occupied by this same event
                occ = occupancy.get(t_str, set())
                if len(occ - {idx}) > 0:
                    candidate_ok = False
                    break

            if candidate_ok:
                suggested_time = candidate
                break

        suggestions.append({
            "event": name,
            "current_time": start_str,
            "duration_hours": dur,
            "suggested_time": suggested_time
        })

    return {
        "has_conflict": True,
        "conflicts": conflict_list,
        "suggestions": suggestions
    }


In [191]:
def run_agent(thread, assistant_id):
    """
    Run the assistant until it finishes.
    If it asks to call tools, we execute generate_schedule()
    and feed results back.
    """
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

        if run_status.status == "completed":
            break

        elif run_status.status == "requires_action":
            tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
            outputs = []

            for call in tool_calls:
                fn = call.function.name
                args = json.loads(call.function.arguments)

                if fn == "load_calendar":
                    result = load_calendar()

                elif fn == "get_cat_fact":
                    result = get_cat_fact()

                elif fn == "generate_schedule":
                    result = generate_schedule(**args)

                elif fn == "analyze_overlaps":
                    result = analyze_overlaps(**args)

                else:
                    result = {"error": f"unknown function {fn}"}

                

                outputs.append({
                    "tool_call_id": call.id,
                    "output": json.dumps(result)
                })

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=outputs
            )

        # else: queued / in_progress → loop again

    return client.beta.threads.messages.list(thread_id=thread.id)


In [192]:
assistant = client.beta.assistants.create(
    name="NaturalLanguageScheduler",
    model="gpt-4.1",
    instructions="""
You are a daily scheduling agent.

USER INPUT:
- The user will describe what they want in natural language, e.g.:
  "Make me a schedule where I have GYM at 18:00 for 2 hours,
   and eat for 1 hour 3 times a day.
   Start at 08:00 and end at 22:00."

YOUR JOB:
1. Infer the constraints:
   - start_time and end_time (HH:MM, 24-hour) from the user's text.
   - If not specified, default to 08:00 and 22:00.

2. Infer the events list:
   - Turn the user's description into a list of objects:
     {"name": <string>, "time": "HH:MM", "duration_hours": <float>}
   - For events like “eat for 1 hour 3 times a day”, DO NOT choose times.
    Instead create:
    {"name": "Eat", "duration_hours": 1.0, "flexible": true}

    If the user wants multiple occurrences (e.g., “eat 3 times”),
    produce that many flexible events:
    {"name": "Eat", "duration_hours": 1.0, "flexible": true}
    {"name": "Eat", "duration_hours": 1.0, "flexible": true}
    {"name": "Eat", "duration_hours": 1.0, "flexible": true}

    Leave placement to generate_schedule; do NOT guess times.
    
- For "GYM at 18:00 for 2 hours", create:
    {"name": "Gym", "time": "18:00", "duration_hours": 2.0}

3. Call the generate_schedule tool with:
   {
     "constraints": {"start_time": "...", "end_time": "..."},
     "events": [ ... ]
   }

4. Then, format the schedule as text in this exact layout:

   Time | Task
   08:00 | FREE TIME
   08:30 | FREE TIME
   09:00 | Eat
   ...

   - One row per line.
   - Exactly one " | " separator per row.
   - Sorted by time.
   
5. AFTER YOU PRINT THE SCHEDULE TABLE:
    - You MUST call the get_cat_fact tool.
    - Then append the cat fact below the table in this format:

    Cat Fact: <fact>

    This is mandatory after every schedule request.


6.CONFLICT POLICY:

Before you call generate_schedule, you MUST call the analyze_overlaps tool
with the same constraints and events you plan to schedule.

- If analyze_overlaps.has_conflict is false:
    - It is safe to call generate_schedule and return the schedule table.

- If analyze_overlaps.has_conflict is true:
    - DO NOT call generate_schedule.
    - Instead, read the "conflicts" and "suggestions" fields.
    - Explain to the user which events overlap and at what times.
    - For each item in "suggestions" that has a non-null suggested_time,
      tell the user something like:

      "Your event 'Eat' at 19:00 (1.0h) conflicts with 'Gym'.
       I suggest moving it to 20:00."

    - Ask the user how they want to adjust their events.
    - Do not produce a full schedule in the same response when conflicts exist.


Do NOT mention internal JSON or tools. The user should just see their schedule.
""",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_cat_fact",
                "description": "Gets a fun cat fact.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_overlaps",
                "description": "Check for overlapping fixed events and suggest alternative times.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "constraints": {
                            "type": "object",
                            "properties": {
                                "start_time": {"type": "string"},
                                "end_time": {"type": "string"}
                            },
                            "required": ["start_time", "end_time"]
                        },
                        "events": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "name": {"type": "string"},
                                    "time": {"type": "string"},
                                    "duration_hours": {"type": "number"},
                                    "flexible": {"type": "boolean"}
                                },
                                "required": ["name"]
                            }
                        }
                    },
                    "required": ["constraints", "events"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "generate_schedule",
                "description": "Create a 30-minute resolution schedule from constraints and events.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "constraints": {
                            "type": "object",
                            "properties": {
                                "start_time": {"type": "string"},
                                "end_time": {"type": "string"}
                            },
                            "required": ["start_time", "end_time"]
                        },
                        "events": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "name": {"type": "string"},
                                    "time": {"type": "string"},
                                    "duration_hours": {"type": "number"}
                                },
                                "required": ["name", "time", "duration_hours"]
                            }
                        }
                    },
                    "required": ["constraints", "events"]
                }
            }
        }
    ]
)

assistant


Assistant(id='asst_xeLLPN6EXley3xqmil1T2y3d', created_at=1764026484, description=None, instructions='\nYou are a daily scheduling agent.\n\nUSER INPUT:\n- The user will describe what they want in natural language, e.g.:\n  "Make me a schedule where I have GYM at 18:00 for 2 hours,\n   and eat for 1 hour 3 times a day.\n   Start at 08:00 and end at 22:00."\n\nYOUR JOB:\n1. Infer the constraints:\n   - start_time and end_time (HH:MM, 24-hour) from the user\'s text.\n   - If not specified, default to 08:00 and 22:00.\n\n2. Infer the events list:\n   - Turn the user\'s description into a list of objects:\n     {"name": <string>, "time": "HH:MM", "duration_hours": <float>}\n   - For events like “eat for 1 hour 3 times a day”, DO NOT choose times.\n    Instead create:\n    {"name": "Eat", "duration_hours": 1.0, "flexible": true}\n\n    If the user wants multiple occurrences (e.g., “eat 3 times”),\n    produce that many flexible events:\n    {"name": "Eat", "duration_hours": 1.0, "flexible": 

INPUT REQUEST HERE!!!!!!!!!!!!!

In [195]:
thread = client.beta.threads.create()

user_request = """
Make me a schedule where I have GYM at 18:00 for 2 hours,
and eat for 1 hour 3 times a day from 0800 1300 and 1700. I also want to dedicate 2 hours of reading time from 2000 and study time from 1000 for 2 hours
Start at 08:00 and end at 22:00.
"""

client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_request
)

messages = run_agent(thread, assistant.id)

for msg in messages.data:
    print(msg.role, ":", msg.content)


C:\Users\ZarifBin.MohdZamrin\AppData\Local\Temp\ipykernel_21812\4009354187.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\ZarifBin.MohdZamrin\AppData\Local\Temp\ipykernel_21812\4009354187.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\ZarifBin.MohdZamrin\AppData\Local\Temp\ipykernel_21812\445245267.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\ZarifBin.MohdZamrin\AppData\Local\Temp\ipykernel_21812\445245267.py:13: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(
C:\Users\ZarifBin.MohdZamrin\AppData\Local\Temp\ipykernel_21812\445245267.py:51: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.be

assistant : [TextContentBlock(text=Text(annotations=[], value='Time | Task\n08:00 | Eat\n08:30 | Eat\n09:00 | FREE TIME\n09:30 | FREE TIME\n10:00 | Study\n10:30 | Study\n11:00 | Study\n11:30 | Study\n12:00 | FREE TIME\n12:30 | FREE TIME\n13:00 | Eat\n13:30 | Eat\n14:00 | FREE TIME\n14:30 | FREE TIME\n15:00 | FREE TIME\n15:30 | FREE TIME\n16:00 | FREE TIME\n16:30 | FREE TIME\n17:00 | Eat\n17:30 | Eat\n18:00 | Gym\n18:30 | Gym\n19:00 | Gym\n19:30 | Gym\n20:00 | Reading\n20:30 | Reading\n21:00 | Reading\n21:30 | Reading\n\nCat Fact: A female cat can be referred to as a molly or a queen, and a male cat is often labeled as a tom.'), type='text')]
user : [TextContentBlock(text=Text(annotations=[], value='\nMake me a schedule where I have GYM at 18:00 for 2 hours,\nand eat for 1 hour 3 times a day from 0800 1300 and 1700. I also want to dedicate 2 hours of reading time from 2000 and study time from 1000 for 2 hours\nStart at 08:00 and end at 22:00.\n'), type='text')]


C:\Users\ZarifBin.MohdZamrin\AppData\Local\Temp\ipykernel_21812\445245267.py:59: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.messages.list(thread_id=thread.id)


In [196]:
assistant_msg = next(m for m in messages.data if m.role == "assistant")
schedule_text = assistant_msg.content[0].text.value
print(schedule_text)  # raw table text

def text_to_schedule_df(text: str) -> pd.DataFrame:
    lines = [ln.strip() for ln in text.splitlines() if " | " in ln]
    if not lines:
        return pd.DataFrame(columns=["Time", "Task"])
    rows = []
    for line in lines[1:]:  # skip header
        parts = [p.strip() for p in line.split(" | ", 1)]
        if len(parts) == 2:
            rows.append({"Time": parts[0], "Task": parts[1]})
    return pd.DataFrame(rows)

schedule_df = text_to_schedule_df(schedule_text)
schedule_df


Time | Task
08:00 | Eat
08:30 | Eat
09:00 | FREE TIME
09:30 | FREE TIME
10:00 | Study
10:30 | Study
11:00 | Study
11:30 | Study
12:00 | FREE TIME
12:30 | FREE TIME
13:00 | Eat
13:30 | Eat
14:00 | FREE TIME
14:30 | FREE TIME
15:00 | FREE TIME
15:30 | FREE TIME
16:00 | FREE TIME
16:30 | FREE TIME
17:00 | Eat
17:30 | Eat
18:00 | Gym
18:30 | Gym
19:00 | Gym
19:30 | Gym
20:00 | Reading
20:30 | Reading
21:00 | Reading
21:30 | Reading

Cat Fact: A female cat can be referred to as a molly or a queen, and a male cat is often labeled as a tom.


,Time,Task
0,08:00,Eat
1,08:30,Eat
2,09:00,FREE TIME
3,09:30,FREE TIME
4,10:00,Study
5,10:30,Study
6,11:00,Study
7,11:30,Study
8,12:00,FREE TIME
9,12:30,FREE TIME
